### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
from IPython.core.display import display, HTML, clear_output
from ipywidgets import Button, Text, Layout, Box, Output
import functools

### Выгрузка данных
Cсылка на API: https://github.com/hhru/api 

In [2]:
# Выбор названий, временного отрезка и опыта

text_list = [
            'Data Scientist',
            'Data Science',
            'Дата Cаентист',
            'Дата Саенс', 
            'Machine Learning',
            'Машинное обучение',
            'ML',
#             'Специалист по machine learning',
#             'Специалист по машинному обучению',
#             'Machine Learning Engineer',
#             'Machine Learning Developer',
#             'Machine Learning Researcher',
#             'Разработчик Machine Learning',
#             'Machine Learning инженер',
#             'ML Engineer',
#             'ML Developer',
#             'ML Researcher',
#             'Разработчик ML',
#             'ML инженер',
            'Data Analyst'
            'Аналитик данных',
            'Специалист по анализу данных'
            ]

text_str = '" or "'.join([i for i in text_list])
text = f'SQL NAME:("{text_str}")'

dt_now = datetime.now().date()
dt_from = dt_now - timedelta(days=7)
date_from = '%s-%s-%s' % (dt_from.year, dt_from.month, dt_from.day)

url_0 = 'https://api.hh.ru/'
url_1 = url_0 + f'vacancies?text={text}&date_from={date_from}&responses_count_enabled=True&per_page=100'

experiences = ['noExperience', 'between1And3']

In [3]:
# Выгрузка данных

request_data = pd.DataFrame()
for exp in experiences:
    page = 0
    url_2 = url_1 + f'&experience={exp}'
    try:
        r = requests.get(url_2 + f'&page={page}')
        data = json.loads(r.text)
        print(exp, ' - ', data['pages'], 'pages')
        items = data['items']
        for page in range(1, data['pages']):
            r = requests.get(url_2 + f'&page={page}')
            data = json.loads(r.text)
            items += data['items']
        df = pd.DataFrame(items)
        df['experience'] = exp
        request_data = pd.concat([request_data, df])
    except Exception as e:
        print(exp, '-', e)
        
request_data.reset_index(drop=True, inplace=True)
print('\nDataFrame shape:', request_data.shape)
pd.set_option('display.max_columns', None);
request_data.sample(3)

noExperience  -  1 pages
between1And3  -  2 pages

DataFrame shape: (114, 30)


,id,premium,name,department,has_test,response_letter_required,area,salary,type,address,response_url,sort_point_distance,published_at,created_at,archived,apply_alternate_url,insider_interview,url,alternate_url,relations,employer,snippet,contacts,schedule,counters,working_days,working_time_intervals,working_time_modes,accept_temporary,experience
88,46705600,False,Data scientist (Predictive Analytics),None,False,False,"{'id': '88', 'name': 'Казань', 'url': 'https:/...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'город Иннополис', 'street': None, 'b...",None,None,2021-10-25T09:27:15+0300,2021-10-25T09:27:15+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/46705600?host=hh.ru,https://hh.ru/vacancy/46705600,[],"{'id': '1160188', 'name': 'Университет Иннопол...",{'requirement': 'Опыт работы с рекуррентными Н...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 4},[],[],[],False,between1And3
43,49000778,False,Middle Machine Learning/Data Science инженер,None,False,False,"{'id': '1004', 'name': 'Могилев', 'url': 'http...",None,"{'id': 'open', 'name': 'Открытая'}",None,None,None,2021-10-28T14:47:25+0300,2021-10-28T14:47:25+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/49000778?host=hh.ru,https://hh.ru/vacancy/49000778,[],"{'id': '1858119', 'name': 'Innowise Group / Фа...","{'requirement': 'Опыт работы с Numpy, Pandas; ...",None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 0},[],[],[],False,between1And3
47,49213609,False,Machine Learning Engineer,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 200000, 'to': 350000, 'currency': 'RU...","{'id': 'open', 'name': 'Открытая'}",None,None,None,2021-10-31T12:05:56+0300,2021-10-31T12:05:56+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/49213609?host=hh.ru,https://hh.ru/vacancy/49213609,[],"{'id': '5465515', 'name': 'Nexus FrontierTech ...",{'requirement': 'Бакалавриат по специальностям...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 11},[],[],[],True,between1And3


### Фильтрация датасета

In [4]:
# Извлечение нужных столбцов и информации из словарей в новый датафрейм

def return_id(x, key='id'):
    try:
        return x[key]
    except:
        return np.nan
    
vacancies = request_data.copy()[['id', 'name', 'published_at', 'url', 'alternate_url', 'experience']]

vacancies['employer'] = request_data['employer'].apply(return_id, key='name')
vacancies['city'] = request_data['area'].apply(return_id, key='name')
vacancies['type'] = request_data['type'].apply(return_id, key='name')
vacancies['schedule'] = request_data['schedule'].apply(return_id, key='name')
vacancies['responses'] = request_data['counters'].apply(return_id, key='responses')
vacancies['salary_from'] = request_data['salary'].apply(return_id, key='from')
vacancies['salary_to'] = request_data['salary'].apply(return_id, key='to')
vacancies['currency'] = request_data['salary'].apply(return_id, key='currency')

In [5]:
# Фильтрация датасета

black_words = ['ведущий', 'старший', 'senior', 'middle', 'teamlead', 'team lead',  'lead', 'руководитель', 
               'мидл', 'миддл', 'сеньор', 'сениор', 'тимлид', 'тим лид',  'главный', 'директор', 'управляющий']
white_words = ['junior', 'juniour', 'джуниор', 'младший', 'стажер', 'стажёр', 'интерн', 'intern']

def check_name(col):
    return (any([i in col.lower() for i in black_words]) &
            (not any([i in col.lower() for i in white_words])))

mask = (
            (
                (vacancies['city']!='Москва') & 
                (vacancies['city']!='Нижний Новгород') & 
                (vacancies['schedule']!='Удаленная работа')
            ) 
            |
            (
                vacancies['type']!='Открытая'
            ) 
            |
            (
                vacancies['name'].apply(check_name)
            )
       )

vacancies.drop(labels=vacancies[mask].index, inplace=True)
vacancies.drop_duplicates(subset=['name', 'employer'], inplace=True)
vacancies.sort_values('published_at', ascending=False, inplace=True)
vacancies.reset_index(drop=True, inplace=True)

print('DataFrame shape:', vacancies.shape)
vacancies.head(5)

DataFrame shape: (48, 14)


,id,name,published_at,url,alternate_url,experience,employer,city,type,schedule,responses,salary_from,salary_to,currency
0,49277183,Бизнес-аналитик (Data Science направление),2021-11-01T10:30:20+0300,https://api.hh.ru/vacancies/49277183?host=hh.ru,https://hh.ru/vacancy/49277183,between1And3,red_mad_robot,Москва,Открытая,Полный день,1,NaN,NaN,NaN
1,48980498,Data scientist,2021-11-01T09:44:32+0300,https://api.hh.ru/vacancies/48980498?host=hh.ru,https://hh.ru/vacancy/48980498,between1And3,"МАГНИТ, Розничная сеть",Москва,Открытая,Удаленная работа,2,NaN,NaN,NaN
2,49172940,Data scientist (CRM),2021-11-01T09:26:10+0300,https://api.hh.ru/vacancies/49172940?host=hh.ru,https://hh.ru/vacancy/49172940,between1And3,"РОСБАНК, Societe Generale Group (Russia)",Москва,Открытая,Полный день,0,NaN,NaN,NaN
3,49218764,Data Scientist (клиентские данные),2021-11-01T09:26:05+0300,https://api.hh.ru/vacancies/49218764?host=hh.ru,https://hh.ru/vacancy/49218764,between1And3,"РОСБАНК, Societe Generale Group (Russia)",Москва,Открытая,Полный день,2,NaN,NaN,NaN
4,48500730,Data Scientist,2021-10-31T17:32:33+0300,https://api.hh.ru/vacancies/48500730?host=hh.ru,https://hh.ru/vacancy/48500730,between1And3,Plarium,Москва,Открытая,Полный день,46,NaN,NaN,NaN


### Выгрузка и просмотр содержимого

In [6]:
# Выгрузка содержимого

vacs = vacancies.head(50)['url'].apply(lambda x: HTML(json.loads(requests.get(x).text)['description']))

In [7]:
# Настройки панели управления

items = [Button(layout=Layout(width='47.5%'), icon='arrow-left'),
         Text(value='', layout=Layout(width='5%')),
         Button(layout=Layout(width='47.5%'), icon='arrow-right')]
box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='100%')
box = Box(children=items, layout=box_layout)
output = Output()
display(box, output)

# Функция для печати содержимого
def print_content(idx):
    clear_output()
    display(HTML('<a href="' + str(vacancies['alternate_url'][idx]) + '" target="_blank">Ссылка на вакансию</a>'))
    display(HTML("<h3>" + str(vacancies['name'][idx]) + ' в ' + str(vacancies['employer'][idx]) + "</h3>"))
    display(vacs[idx])
    print('\nДата публикации:', vacancies['published_at'][idx][:10], ' Количество откликов:', vacancies['responses'][idx])

# Функция для реакции на клик
def on_clicked(b, direction=0):
    with output:
        try:
            global vac_idx
            try:
                vac_idx = (vac_idx + direction)%len(vacs)
            except:
                vac_idx = 0
            box.children[1].value = str(vac_idx)
            print_content(vac_idx)
        except Exception as e:
            clear_output()
            print(e)
    
# Функция для реакции на ввод текста
def on_submitted(b):
    with output:
        try:
            global vac_idx
            val = int(box.children[1].value)
            if val >= len(vacs):
                vac_idx = len(vacs) - 1
            elif val < 0:
                vac_idx = 0
            else:
                vac_idx = val
            box.children[1].value = str(vac_idx)
            print_content(vac_idx)
        except Exception as e:
            clear_output()
            print(e)

box.children[0].on_click(functools.partial(on_clicked, direction=-1))
box.children[1].on_submit(on_submitted)
box.children[2].on_click(functools.partial(on_clicked, direction=1))

Box(children=(Button(icon='arrow-left', layout=Layout(width='47.5%'), style=ButtonStyle()), Text(value='', lay…

Output()